In [58]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [2]:
import numpy as np
from matplotlib import pyplot as plt
#import csv
import math
import pandas

def plot_log(filename, show=True):

    data = pandas.read_csv(filename)

    fig = plt.figure(figsize=(4,6))
    fig.subplots_adjust(top=0.95, bottom=0.05, right=0.95)
    fig.add_subplot(211)
    for key in data.keys():
        if key.find('loss') >= 0 and not key.find('val') >= 0:  # training loss
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training loss')

    fig.add_subplot(212)
    for key in data.keys():
        if key.find('acc') >= 0:  # acc
            plt.plot(data['epoch'].values, data[key].values, label=key)
    plt.legend()
    plt.title('Training and validation accuracy')

    fig.savefig('result/log.png')
    if show:
        plt.show()


def combine_images(generated_images, height=None, width=None):
    num = generated_images.shape[0]
    if width is None and height is None:
        width = int(math.sqrt(num))
        height = int(math.ceil(float(num)/width))
    elif width is not None and height is None:  # height not given
        height = int(math.ceil(float(num)/width))
    elif height is not None and width is None:  # width not given
        width = int(math.ceil(float(num)/height))

    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1]),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, 0]
    return image


In [3]:
import keras.backend as K
import tensorflow as tf
from keras import initializers, layers


class Length(layers.Layer):
    """
    Compute the length of vectors.
    This is used to compute a Tensor that has the same shape with y_true in margin_loss.
    Using this layer as model's output can directly predict labels by using `y_pred = np.argmax(model.predict(x), 1)`

    Args:
        inputs: shape=[None, num_vectors, dim_vector]

    Returns:
        output: shape=[None, num_vectors]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

    def get_config(self):
        config = super(Length, self).get_config()
        return config


class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, num_capsule, dim_vector] either by the capsule with max length or by an additional input mask.
    Except the max-length capsule (or specified capsule), all vectors are masked to zeros.
    Then flatten the masked Tensor.

    For example:
        ```
        x = keras.layers.Input(shape=[8, 3, 2])  # batch_size=8, each sample contains 3 capsules with dim_vector=2
        y = keras.layers.Input(shape=[8, 3])  # True labels. 8 samples, 3 classes, one-hot coding.
        out = Mask()(x)  # out.shape=[8, 6]
        # or
        out2 = Mask()([x, y])  # out2.shape=[8,6]. Masked with true labels y. Of course y can also be manipulated.
        ```
    """
    def call(self, inputs, **kwargs):
        if type(inputs) is list:  # true label is provided with shape = [None, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of capsules. Mainly used for prediction
            # compute lengths of capsules
            x = K.sqrt(K.sum(K.square(inputs), -1))
            # generate the mask which is a one-hot code.
            # mask.shape=[None, n_classes]=[None, num_capsule]
            mask = K.one_hot(indices=K.argmax(x, 1), num_classes=x.get_shape().as_list()[1])

        # inputs.shape=[None, num_capsule, dim_capsule]
        # mask.shape=[None, num_capsule]
        # masked.shape=[None, num_capsule * dim_capsule]
        masked = K.batch_flatten(inputs * K.expand_dims(mask, -1))
        return masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][1] * input_shape[0][2]])
        else:  # no true label provided
            return tuple([None, input_shape[1] * input_shape[2]])

    def get_config(self):
        config = super(Mask, self).get_config()
        return config


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0

    Args:
        vectors: some vectors to be squashed, N-dim tensor
        axis: the axis to squash
    Returns:
        a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm + K.epsilon())
    return scale * vectors

class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        inputs_expand = K.expand_dims(inputs, 1)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

        # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
        # x.shape=[num_capsule, input_num_capsule, input_dim_capsule]
        # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)

        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        # b.shape = [None, self.num_capsule, self.input_num_capsule].
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            # c.shape=[batch_size, num_capsule, input_num_capsule]
            c = tf.nn.softmax(b, dim=1)

            # c.shape =  [batch_size, num_capsule, input_num_capsule]
            # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
            # The first two dimensions as `batch` dimension,
            # then matmal: [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule].
            # outputs.shape=[None, num_capsule, dim_capsule]
            outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]

            if i < self.routings - 1:
                # outputs.shape =  [None, num_capsule, dim_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension,
                # then matmal: [dim_capsule] x [input_num_capsule, dim_capsule]^T -> [input_num_capsule].
                # b.shape=[batch_size, num_capsule, input_num_capsule]
                b += K.batch_dot(outputs, inputs_hat, [2, 3])
        # End: Routing algorithm -----------------------------------------------------------------------#

        return outputs

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(CapsuleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def PrimaryCap(inputs, dim_capsule, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules

    Args:
        inputs: 4D tensor, shape=[None, width, height, channels]
        dim_capsule: the dim of the output vector of capsule
        n_channels: the number of types of capsules
    Returns:
        output tensor, shape=[None, num_capsule, dim_capsule]
    """
    output = layers.Conv2D(filters=dim_capsule*n_channels, kernel_size=kernel_size, strides=strides, padding=padding,
                           name='primarycap_conv2d')(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_capsule], name='primarycap_reshape')(output)
    return layers.Lambda(squash, name='primarycap_squash')(outputs)


In [4]:
class InputsHatLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_capsule] and output shape = \
    [None, num_capsule, dim_capsule]. For Dense Layer, input_dim_capsule = dim_capsule = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_capsule: dimension of the output vectors of the capsules in this layer
    :param routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_capsule, routings=3,
                 kernel_initializer='glorot_uniform',
                 **kwargs):
        super(InputsHatLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_initializer = initializers.get(kernel_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_capsule]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_capsule = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.num_capsule, self.input_num_capsule,
                                        self.dim_capsule, self.input_dim_capsule],
                                 initializer=self.kernel_initializer,
                                 name='W')

        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_capsule]
        # inputs_expand.shape=[None, 1, input_num_capsule, input_dim_capsule]
        inputs_expand = K.expand_dims(inputs, 1)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # inputs_tiled.shape=[None, num_capsule, input_num_capsule, input_dim_capsule]
        inputs_tiled = K.tile(inputs_expand, [1, self.num_capsule, 1, 1])

        # Compute `inputs * W` by scanning inputs_tiled on dimension 0.
        # x.shape=[num_capsule, input_num_capsule, input_dim_capsule]
        # W.shape=[num_capsule, input_num_capsule, dim_capsule, input_dim_capsule]
        # Regard the first two dimensions as `batch` dimension,
        # then matmul: [input_dim_capsule] x [dim_capsule, input_dim_capsule]^T -> [dim_capsule].
        # inputs_hat.shape = [None, num_capsule, input_num_capsule, dim_capsule]
        inputs_hat = K.map_fn(lambda x: K.batch_dot(x, self.W, [2, 3]), elems=inputs_tiled)

        # Begin: Routing algorithm ---------------------------------------------------------------------#
        # The prior for coupling coefficient, initialized as zeros.
        # b.shape = [None, self.num_capsule, self.input_num_capsule].
        b = tf.zeros(shape=[K.shape(inputs_hat)[0], self.num_capsule, self.input_num_capsule])

        assert self.routings > 0, 'The routings should be > 0.'
        for i in range(self.routings):
            # c.shape=[batch_size, num_capsule, input_num_capsule]
            c = tf.nn.softmax(b, dim=1)

            # c.shape =  [batch_size, num_capsule, input_num_capsule]
            # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
            # The first two dimensions as `batch` dimension,
            # then matmal: [input_num_capsule] x [input_num_capsule, dim_capsule] -> [dim_capsule].
            # outputs.shape=[None, num_capsule, dim_capsule]
            outputs = squash(K.batch_dot(c, inputs_hat, [2, 2]))  # [None, 10, 16]

            if i < self.routings - 1:
                # outputs.shape =  [None, num_capsule, dim_capsule]
                # inputs_hat.shape=[None, num_capsule, input_num_capsule, dim_capsule]
                # The first two dimensions as `batch` dimension,
                # then matmal: [dim_capsule] x [input_num_capsule, dim_capsule]^T -> [input_num_capsule].
                # b.shape=[batch_size, num_capsule, input_num_capsule]
                b += K.batch_dot(outputs, inputs_hat, [2, 3])
        # End: Routing algorithm -----------------------------------------------------------------------#

        return inputs_hat

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, input_shape[1], self.dim_capsule])

    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(CapsuleLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from PIL import Image

K.set_image_data_format('channels_last')

def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.

    Args:
        y_true: [None, n_classes]
        y_pred: [None, num_capsule]

    Returns
        a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))


def train(model, data, args):
    """
    Training a CapsuleNet

    Args:
        model: the CapsuleNet model
        data: a tuple containing training and testing data, like `((x_train, y_train), (x_test, y_test))`
        args: arguments

    Returns:
        The trained model
    """
    # unpacking the data
    (x_train, y_train), (x_test, y_test) = data

    # callbacks
    log = callbacks.CSVLogger(args.save_dir + '/log.csv', separator=',', append=True)
    tb = callbacks.TensorBoard(log_dir=args.save_dir + '/tensorboard-logs',
                               batch_size=args.batch_size, histogram_freq=int(args.debug))
    checkpoint = callbacks.ModelCheckpoint(args.save_dir + '/weights-{epoch:02d}.h5', monitor='val_capsnet_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
    lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: args.lr * (args.lr_decay ** epoch))

    # compile the model
    model.compile(optimizer=optimizers.Adam(lr=args.lr),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., args.lam_recon],
                  metrics={'capsnet': 'accuracy'})

    """
    # Training without data augmentation:
    model.fit([x_train, y_train], [y_train, x_train], batch_size=args.batch_size, epochs=args.epochs,
              validation_data=[[x_test, y_test], [y_test, x_test]], callbacks=[log, tb, checkpoint, lr_decay])
    """

    # Begin: Training with data augmentation ---------------------------------------------------------------------#
    def train_generator(x, y, batch_size, shift_fraction=0.):
        train_datagen = ImageDataGenerator(width_shift_range=shift_fraction,
                                           height_shift_range=shift_fraction)  # shift up to 2 pixel for MNIST
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch], [y_batch, x_batch])

    # Training with data augmentation. If shift_fraction=0., also no augmentation.
    model.fit_generator(generator=train_generator(x_train, y_train, args.batch_size, args.shift_fraction),
                        steps_per_epoch=int(y_train.shape[0] / args.batch_size),
                        initial_epoch=args.initial_epoch,
                        epochs=args.epochs,
                        validation_data=[[x_test, y_test], [y_test, x_test]],
                        callbacks=[log, tb, checkpoint, lr_decay])
    # End: Training with data augmentation -----------------------------------------------------------------------#

    model.save_weights(args.save_dir + '/trained_model.h5')
    print('Trained model saved to \'%s/trained_model.h5\'' % args.save_dir)

    from utils import plot_log
    plot_log(args.save_dir + '/log.csv', show=True)

    return model


def test(model, data, args):
    x_test, y_test = data
    y_pred, x_recon = model.predict(x_test, batch_size=100)
    print('-'*30 + 'Begin: test' + '-'*30)
    print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])

    img = combine_images(np.concatenate([x_test[:50],x_recon[:50]]))
    image = img * 255
    Image.fromarray(image.astype(np.uint8)).save(args.save_dir + "/real_and_recon.png")
    print()
    print('Reconstructed images are saved to %s/real_and_recon.png' % args.save_dir)
    print('-' * 30 + 'End: test' + '-' * 30)
    plt.imshow(plt.imread(args.save_dir + "/real_and_recon.png"))
    plt.show()


def manipulate_latent(model, data, args):
    print('-'*30 + 'Begin: manipulate' + '-'*30)
    x_test, y_test = data
    index = np.argmax(y_test, 1) == args.digit
    number = np.random.choice(sum(index), size=1, replace=False) # 汎用性のため変更
    x, y = x_test[index][number], y_test[index][number] # numberはndarrayなので次元は減らない
    # x, y = np.expand_dims(x, 0), np.expand_dims(y, 0) 上の変更により不要
    noise = np.zeros([1, 10, 16])
    x_recons = []
    for dim in range(16):
        for r in [-0.25, -0.2, -0.15, -0.1, -0.05, 0, 0.05, 0.1, 0.15, 0.2, 0.25]:
            tmp = np.copy(noise)
            tmp[:,:,dim] = r
            x_recon = model.predict([x, y, tmp])
            x_recons.append(x_recon)

    x_recons = np.concatenate(x_recons)

    img = combine_images(x_recons, height=16)
    image = img*255
    Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/manipulate-%d.png' % args.digit)
    print('manipulated result saved to %s/manipulate-%d.png' % (args.save_dir, args.digit))
    print('-' * 30 + 'End: manipulate' + '-' * 30)


def load_mnist():
    # the data, shuffled and split between train and test sets
    from keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.
    y_train = to_categorical(y_train.astype('float32'))
    y_test = to_categorical(y_test.astype('float32'))
    return (x_train, y_train), (x_test, y_test)


In [6]:
import os
import argparse
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks

# setting the hyper parameters
parser = argparse.ArgumentParser(description="Capsule Network on MNIST.")
parser.add_argument('--initial_epoch', default=0, type=int)
parser.add_argument('--epochs', default=50, type=int)
parser.add_argument('--batch_size', default=100, type=int)
parser.add_argument('--lr', default=0.001, type=float,
                    help="Initial learning rate")
parser.add_argument('--lr_decay', default=0.9, type=float,
                    help="The value multiplied by lr at each epoch. Set a larger value for larger epochs")
parser.add_argument('--lam_recon', default=0.392, type=float,
                    help="The coefficient for the loss of decoder")
parser.add_argument('-r', '--routings', default=3, type=int,
                    help="Number of iterations used in routing algorithm. should > 0")
parser.add_argument('--shift_fraction', default=0.1, type=float,
                    help="Fraction of pixels to shift at most in each direction.")
parser.add_argument('--debug', action='store_true',
                    help="Save weights by TensorBoard")
parser.add_argument('--save_dir', default='./result')
parser.add_argument('-t', '--testing', action='store_true',
                    help="Test the trained model on testing dataset")
parser.add_argument('--digit', default=5, type=int,
                    help="Digit to manipulate")
parser.add_argument('-w', '--weights', default=None,
                    help="The path of the saved weights. Should be specified when testing")
args = parser.parse_args('-t -w result_20181025/trained_model.h5'.split())
print(args)

Namespace(batch_size=100, debug=False, digit=5, epochs=50, initial_epoch=0, lam_recon=0.392, lr=0.001, lr_decay=0.9, routings=3, save_dir='./result', shift_fraction=0.1, testing=True, weights='result_20181025/trained_model.h5')


In [7]:
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

# load data
(x_train, y_train), (x_test, y_test) = load_mnist()

In [8]:
input_shape = x_train.shape[1:]
n_class = len(np.unique(np.argmax(y_train, 1)))
routings = args.routings

x = layers.Input(shape=input_shape)

# Layer 1: Just a conventional Conv2D layer
conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

# Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

# Layer 3: Capsule layer. Routing algorithm works here.
digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                         name='digitcaps')(primarycaps)

# Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
# If using tensorflow, this will not be necessary. :)
out_caps = Length(name='capsnet')(digitcaps)

# Decoder network.
y = layers.Input(shape=(n_class,))
masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

# Shared Decoder model in training and prediction
decoder = models.Sequential(name='decoder')
decoder.add(layers.Dense(512, activation='relu', input_dim=n_class*16))
decoder.add(layers.Dense(1024, activation='relu'))
decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

# Models for training and evaluation (prediction)
train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
eval_model = models.Model(x, [out_caps, decoder(masked)])

# manipulate model
noise = layers.Input(shape=(n_class, 16))
noised_digitcaps = layers.Add()([digitcaps, noise])
masked_noised_y = Mask()([noised_digitcaps, y])
manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))

# primarycapsを出力するモデル
pricaps_model = models.Model(x, primarycaps)

# xからinputs_hatを出力するモデル
inputs_hat_layer = InputsHatLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                         name='inputs_hat')(primarycaps)
inputs_hat_model = models.Model(x, inputs_hat_layer)

# pricapsからinput_hatを出力するモデル
primarycaps_mod = layers.Input(shape=(1152, 8))
inputs_hat_mod = InputsHatLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                         name='inputs_hat_mod')(primarycaps_mod)
inputs_hat_mod_model = models.Model(primarycaps_mod, inputs_hat_mod)

# 弄ったdigitcapsから画像を再構成するモデル
ex_digitcaps= layers.Input(shape=(10, 16))
ex_masked_by_y = Mask()([ex_digitcaps, y])
ex_manipulate_model = models.Model([ex_digitcaps, y], decoder(ex_masked_by_y))

Instructions for updating:
dim is deprecated, use axis instead


In [9]:
# 重みの読み込み
train_model.load_weights(args.weights)
inputs_hat_model.set_weights(train_model.get_weights()[0:5])
inputs_hat_mod_model.set_weights(train_model.get_weights()[4:5])
ex_manipulate_model.set_weights(train_model.get_weights()[5:11])

In [10]:
# 重みのshapeを確認
for i in range(len(train_model.get_weights())):
    print(train_model.get_weights()[i].shape)

(9, 9, 1, 256)
(256,)
(9, 9, 256, 256)
(256,)
(10, 1152, 16, 8)
(160, 512)
(512,)
(512, 1024)
(1024,)
(1024, 784)
(784,)


In [26]:
digit = 3
index = np.argmax(y_test, 1) == digit
x_test_first, y_test_first = x_test[index][1:2], y_test[index][1:2]
print(x_test_first.shape)

(1, 28, 28, 1)


In [21]:
# テストデータの画像化
for digit in range(10):
    index = np.argmax(y_test, 1) == digit
    x_test_first, y_test_first = x_test[index][1:2], y_test[index][1:2]
    image = x_test_first.reshape(28, 28)
    image = image * 255
    Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_test_sample_%d.png' % digit)

In [25]:
# テストデータを回転して画像化
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

for digit in range(10):
    index = np.argmax(y_test, 1) == digit
    x_test_first, y_test_first = x_test[index][1:2], y_test[index][1:2]
    image = x_test_first.reshape(28, 28)
    image = image * 255
    image = Image.fromarray(image.astype(np.uint8))
    image = image.rotate(90, expand=True)
    image = crop_center(image, 28, 28)
    image.save(args.save_dir + '/x_test_sample_%d_rotate_90.png' % digit)

In [ ]:
# 再構成画像の生成
index = np.argmax(y_test, 1) == 3
x_test_first, y_test_first = x_test[index][1:2], y_test[index][1:2]

y_pred, x_recon_true =eval_model.predict(x_test_first)
image = x_recon_true.reshape(28, 28)
image = image * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon_default.png')

In [32]:
# 回転させた画像を再構成
digit = 3
x_test_first = np.array(Image.open(args.save_dir + '/x_test_sample_%d_rotate_-45.png' % digit)).reshape(1, 28, 28, 1)

# 再構成画像の生成
y_pred, x_recon_true =eval_model.predict(x_test_first)
image = x_recon_true.reshape(28, 28)
image = image * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon_%d_rotate_-45.png' % digit)

In [ ]:
inputs_hat = inputs_hat_model.predict(x_test_first)
print(inputs_hat.shape)
print(inputs_hat[0][3][967])

In [ ]:
inputs_hat_tensor = K.variable(inputs_hat)
b = K.variable(np.zeros((1, 10, 1152), dtype=np.float32))

c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])

c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])

c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
c_test_first = K.eval(c)

In [ ]:
print(c_test_first.shape)
indices = np.argsort(c_test_first[0][3])[::-1][0:10]
print(indices)
print(c_test_first[0][3][indices])
print(np.sort(c_test_first[0][3])[::-1][0:10])

In [ ]:
np.sum(c_test_first[0], 1)

In [ ]:
pricaps = pricaps_model.predict(x_test_first)
print(pricaps.shape)
pricaps_2 = pricaps.copy()
pricaps_2[0][967][0] = pricaps_2[0][967][0] + 1.0
print(pricaps[0][967])
print(pricaps_2[0][967])

In [ ]:
inputs_hat_2 = inputs_hat_mod_model.predict(pricaps)
print(inputs_hat_2.shape)
print(inputs_hat_2[0][4][967])

In [ ]:
inputs_hat_2 = inputs_hat_mod_model.predict(pricaps_2)
print(inputs_hat_2.shape)
print(inputs_hat_2[0][4][967])

In [ ]:
inputs_hat_2_tensor = K.variable(inputs_hat_2)
outputs_2 = squash(K.batch_dot(c, inputs_hat_2_tensor, [2, 2]))
output_2 = K.eval(outputs_2)
print(output_2.shape)

In [ ]:
a = inputs_hat_mod_model.predict(pricaps)
a_tensor = K.variable(a)
outputs_a = squash(K.batch_dot(c, a_tensor, [2, 2]))
outputs_a = K.eval(outputs_a)

b = inputs_hat_mod_model.predict(pricaps_2)
b_tensor = K.variable(b)
outputs_b = squash(K.batch_dot(c, b_tensor, [2, 2]))
outputs_b = K.eval(outputs_b)


x_recon_a = ex_manipulate_model.predict([outputs_a, y_test_first])
image = x_recon_a.reshape(28, 28)
image = image * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon_a.png')

x_recon_b = ex_manipulate_model.predict([outputs_b, y_test_first])
image = x_recon_b.reshape(28, 28)
image = image * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon_b.png')

In [ ]:
x_recon = ex_manipulate_model.predict([output_2, y_test_first])
x_recon.shape
image = x_recon.reshape(28, 28)
image = image * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon2.png')

In [11]:
print('-'*30 + 'Begin: manipulate' + '-'*30)

digit = 3
print('digit: %d' % digit)

# テストデータの選択
index = np.argmax(y_test, 1) == digit
i = 1
x_test_first, y_test_first = x_test[index][i:i+1], y_test[index][i:i+1]

inputs_hat = inputs_hat_model.predict(x_test_first)

# Dynamic Routing
inputs_hat_tensor = K.variable(inputs_hat)
b = K.variable(np.zeros((1, 10, 1152), dtype=np.float32))
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
c_test_first = K.eval(c)

indices = np.argsort(c_test_first[0][digit])[::-1][0:10]
print(indices)
print(c_test_first[0][digit][indices])
print(np.sort(c_test_first[0][digit])[::-1][0:10])

------------------------------Begin: manipulate------------------------------
digit: 3
[967  22 999 215 367 992  54 413 960 439]
[0.90232503 0.28612682 0.24360433 0.16516985 0.16434944 0.15771644
 0.15512717 0.14990121 0.14651932 0.1409263 ]
[0.90232503 0.28612682 0.24360433 0.16516985 0.16434944 0.15771644
 0.15512717 0.14990121 0.14651932 0.1409263 ]
967 0.90232503



|digit|i  |c_top     |
|:----|:--|:---------| 
|0    |3  |     |
|1    |3  ||
|2    |4  |0.77116305|
|3    |1  |0.90232503|
|4    |4  |0.8352959 |
|5    |3  |0.76386034|
|6    |3  ||
|7    |3  ||
|8    |3  |
|9    |3  |

In [14]:
# Pricapsの操作
print('-'*30 + 'Begin: manipulate' + '-'*30)

digit = 3
print('digit: %d' % digit)

# テストデータの選択
index = np.argmax(y_test, 1) == digit

i = 1
x_test_first, y_test_first = x_test[index][i:i+1], y_test[index][i:i+1]

inputs_hat = inputs_hat_model.predict(x_test_first)

# Dynamic Routing
inputs_hat_tensor = K.variable(inputs_hat)
b = K.variable(np.zeros((1, 10, 1152), dtype=np.float32))
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
c_test_first = K.eval(c)

indices = np.argsort(c_test_first[0][digit])[::-1][0:10]
print(indices)
print(c_test_first[0][digit][indices])
print(np.sort(c_test_first[0][digit])[::-1][0:10])

top = 1
index = indices[top]
print(index, c_test_first[0][digit][index])

pricaps = pricaps_model.predict(x_test_first)
print(pricaps[0][index][:])

x_recons = []
for dim in range(8):
    print('dim: %d' % dim)
    
    pricaps_2 = pricaps.copy()
    
    pricaps_2[0][index][:] = 0
    pricaps_2[0][index][dim] = -1.0

    inputs_hat_2 = inputs_hat_mod_model.predict(pricaps_2)
    inputs_hat_2_tensor = K.variable(inputs_hat_2)

    c_2 = np.copy(c_test_first)
    c_2[0][:][:] = 0
    c_2[0][digit][index] = 115.2
    c_2_tensor = K.variable(c_2)

    digitcaps_2 = squash(K.batch_dot(c_2_tensor, inputs_hat_2_tensor, [2, 2]))
    digitcaps_2 = K.eval(digitcaps_2)

    x_recon_2 = ex_manipulate_model.predict([digitcaps_2, y_test_first])
    x_recons.append(x_recon_2)
    
    for r in [-1.0, -0.75, -0.5, -0.25, 0., 0.25, 0.5, 0.75, 1.0]:
        pricaps_2 = pricaps.copy()
        
        pricaps_2[0][index][dim] = pricaps_2[0][index][dim] + r
        #pricaps_2[0][index][:] = 0
        #pricaps_2[0][index][dim] = r
        
        inputs_hat_2 = inputs_hat_mod_model.predict(pricaps_2)
        inputs_hat_2_tensor = K.variable(inputs_hat_2)

        c_2 = np.copy(c_test_first)
        c_2[0][:][:] = 0
        c_2[0][digit][index] = 115.2
        c_2_tensor = K.variable(c_2)

        digitcaps_2 = squash(K.batch_dot(c_2_tensor, inputs_hat_2_tensor, [2, 2]))
        digitcaps_2 = K.eval(digitcaps_2)

        x_recon_2 = ex_manipulate_model.predict([digitcaps_2, y_test_first])
        x_recons.append(x_recon_2)

        """digitcaps_3 = squash(K.batch_dot(c, inputs_hat_2_tensor, [2, 2]))
        digitcaps_3 = K.eval(digitcaps_3)

        x_recon_3 = ex_manipulate_model.predict([digitcaps_3, y_test_first])
        x_recons.append(x_recon_3)"""
        
    pricaps_2 = pricaps.copy()
    
    pricaps_2[0][index][:] = 0
    pricaps_2[0][index][dim] = 1.0

    inputs_hat_2 = inputs_hat_mod_model.predict(pricaps_2)
    inputs_hat_2_tensor = K.variable(inputs_hat_2)

    c_2 = np.copy(c_test_first)
    c_2[0][:][:] = 0
    c_2[0][digit][index] = 115.2
    c_2_tensor = K.variable(c_2)

    digitcaps_2 = squash(K.batch_dot(c_2_tensor, inputs_hat_2_tensor, [2, 2]))
    digitcaps_2 = K.eval(digitcaps_2)

    x_recon_2 = ex_manipulate_model.predict([digitcaps_2, y_test_first])
    x_recons.append(x_recon_2)

x_recons = np.concatenate(x_recons)

img = combine_images(x_recons, height=8)
image = img * 255
Image.fromarray(image.astype(np.uint8)).save(args.save_dir + '/x_recon_digit%d_pricaps_manipulated.png' % digit)
print('result saved')

print('-' * 30 + 'End: manipulate' + '-' * 30)

------------------------------Begin: manipulate------------------------------
digit: 3
[967  22 999 215 367 992  54 413 960 439]
[0.90232503 0.28612682 0.24360433 0.16516985 0.16434944 0.15771644
 0.15512717 0.14990121 0.14651932 0.1409263 ]
[0.90232503 0.28612682 0.24360433 0.16516985 0.16434944 0.15771644
 0.15512717 0.14990121 0.14651932 0.1409263 ]
967 0.90232503
[ 0.28508848 -0.23917827  0.33855495 -0.2590488  -0.49731117 -0.4501336
 -0.3081172  -0.34669688]
dim: 0
dim: 1
dim: 2
dim: 3
dim: 4
dim: 5
dim: 6
dim: 7
result saved
------------------------------End: manipulate------------------------------


In [28]:
# 回転させた画像を入れてみる

print('-'*30 + 'Begin: manipulate' + '-'*30)

digit = 3
print('digit: %d' % digit)

# テストデータの選択
index = np.argmax(y_test, 1) == digit
i = 1
x_test_first, y_test_first = x_test[index][i:i+1], y_test[index][i:i+1]

# x_test_firstを自作画像に置き換える
x_test_first = np.array(Image.open(args.save_dir + '/x_test_sample_%d_rotate_-45.png' % digit))
print(x_test_first.shape)

------------------------------Begin: manipulate------------------------------
digit: 3
(28, 28)


In [19]:
# 回転させた画像を入れてみる

print('-'*30 + 'Begin: manipulate' + '-'*30)

digit = 3
print('digit: %d' % digit)

# x_test_firstを自作画像に置き換える
x_test_first = np.array(Image.open(args.save_dir + '/x_test_sample_%d_rotate_-90.png' % digit)).reshape(1, 28, 28, 1)

inputs_hat = inputs_hat_model.predict(x_test_first)

# Dynamic Routing
inputs_hat_tensor = K.variable(inputs_hat)
b = K.variable(np.zeros((1, 10, 1152), dtype=np.float32))
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
b = b + K.batch_dot(outputs, inputs_hat_tensor, [2, 3])
c = tf.nn.softmax(b, dim=1)
outputs = squash(K.batch_dot(c, inputs_hat_tensor, [2, 2]))
c_test_first = K.eval(c)

indices = np.argsort(c_test_first[0][digit])[::-1][0:10]
print(indices)
print(c_test_first[0][digit][indices])
print(np.sort(c_test_first[0][digit])[::-1][0:10])
index = indices[0]
print(index, c_test_first[0][digit][index])

pricaps = pricaps_model.predict(x_test_first)
print(pricaps[0][index][:])

print('-' * 30 + 'End: manipulate' + '-' * 30)

------------------------------Begin: manipulate------------------------------
digit: 3
[ 967  999   22   54  324 1149  548  143   73 1080]
[0.12228876 0.10589919 0.10421858 0.10248243 0.10196219 0.1019311
 0.1016928  0.10169145 0.10164344 0.10161397]
[0.12228876 0.10589919 0.10421858 0.10248243 0.10196219 0.1019311
 0.1016928  0.10169145 0.10164344 0.10161397]
967 0.12228876
[ 0.41361457 -0.23104525  0.3173196  -0.040114   -0.33156642 -0.22718273
  0.2226019  -0.67980886]
------------------------------End: manipulate------------------------------


In [23]:
a = np.array([ 0.28508848, -0.23917827,  0.33855495, -0.2590488,  -0.49731117, -0.4501336, -0.3081172,  -0.34669688])
b = np.array([ 0.34123582, -0.32716322,  0.3793515,  -0.07650059, -0.39103946, -0.42224914, -0.19171607, -0.508725 ])
c = np.array([ 0.41361457, -0.23104525,  0.3173196,  -0.040114,   -0.33156642, -0.22718273,  0.2226019,  -0.67980886])
print(abs(a - b))
print(abs(a - c))

[0.05614734 0.08798495 0.04079655 0.18254821 0.10627171 0.02788446
 0.11640113 0.16202812]
[0.12852609 0.00813302 0.02123535 0.2189348  0.16574475 0.22295087
 0.5307191  0.33311198]


In [24]:
# ソフトマックス関数
def softmax(a):
    # 一番大きい値を取得
    c = np.max(a)
    # 各要素から一番大きな値を引く（オーバーフロー対策）
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    # 要素の値/全体の要素の合計
    y = exp_a / sum_exp_a

    return y 


a = [23.0, 0.94, 5.46]
print (softmax(a))

[9.99999976e-01 2.62702205e-10 2.41254141e-08]


In [84]:
np.random.seed(3)
hoge = np.random.randn(1,2,4)
hoge = K.variable(hoge)
K.eval(hoge)

array([[[ 1.7886285 ,  0.43650985,  0.09649747, -1.8634927 ],
        [-0.27738822, -0.35475898, -0.08274148, -0.6270007 ]]],
      dtype=float32)

In [92]:
fuga = tf.where(hoge >= 0.5, hoge, K.zeros(shape=(1,2,4))
)
K.eval(fuga)

array([[[1.7886285, 0.       , 0.       , 0.       ],
        [0.       , 0.       , 0.       , 0.       ]]], dtype=float32)

In [91]:
K.eval(hoge > 0.5)

array([[[ True, False, False, False],
        [False, False, False, False]]])

In [93]:
a = tf.zeros((1,2))

In [100]:
c = K.eval(K.shape(a))

In [101]:
b = tf.zeros(c)

In [103]:
K.eval(b)

array([[0., 0.]], dtype=float32)